
## A Gentle Introduction to use One-Class SVM for Anomaly Detection
#### Author Nagdev Amruthnath
Date: 1/9/2019

#### Citation Info
If you are using this for your research, please use the following for citation.

Amruthnath, Nagdev, and Tarun Gupta. "A research study on unsupervised machine learning algorithms for early fault detection in predictive maintenance." In 2018 5th International Conference on Industrial Engineering and Applications (ICIEA), pp. 355-361. IEEE, 2018.

#### Disclaimer
This is a tutorial for performing fault detection using machine learning. You this code at your own risk. I do not gurantee that this would work as shown below. If you have any suggestions please branch this project.

### Load Libraries

In [9]:
options(warn=-1)

# load libraries
library(dplyr)
library(e1071)
library(caret)


### Load data
Here we are using data from a bench press. There are total of four different states in this machine and they are split into four different csv files. We need to load the data first. In the data time represents the time between samples, ax is the acceleration on x axis, ay is the acceleration on y axis, az is the acceleration on z axis and at is the G's. The data was collected at sample rate of 100hz.

Four different states of the machine were collected

    1. Nothing attached to drill press
    2. Wooden base attached to drill press
    3. Imbalance created by adding weight to one end of wooden base
    4. Imbalacne created by adding weight to two ends of wooden base.

In [2]:
setwd("/home/")
#read csv files
file1 = read.csv("dry run.csv", sep=",", header =T)
file2 = read.csv("base.csv", sep=",", header =T)
file3 = read.csv("imbalance 1.csv", sep=",", header =T)
file4 = read.csv("imbalance 2.csv", sep=",", header =T)
head(file1)

time,ax,ay,az,aT
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.002,-0.3246,0.2748,0.1502,0.451
0.009,0.6020,-0.1900,-0.3227,0.709
0.019,0.9787,0.3258,0.0124,1.032
0.027,0.6141,-0.4179,0.0471,0.744
0.038,-0.3218,-0.6389,-0.4259,0.833
0.047,-0.3607,0.1332,-0.1291,0.406


We can look at the summary of each file using summary function in R. Below, we can observe that 66 seconds long data is available. We also have min, max and mean for each of the variables.

In [3]:
# summary of each file
summary(file2)

      time               ax                  ay                  az          
 Min.   :  0.004   Min.   :-1.402700   Min.   :-1.693300   Min.   :-3.18930  
 1st Qu.: 27.005   1st Qu.:-0.311100   1st Qu.:-0.429600   1st Qu.:-0.57337  
 Median : 54.142   Median : 0.015100   Median :-0.010700   Median :-0.11835  
 Mean   : 54.086   Mean   : 0.005385   Mean   :-0.002534   Mean   :-0.09105  
 3rd Qu.: 81.146   3rd Qu.: 0.314800   3rd Qu.: 0.419475   3rd Qu.: 0.34815  
 Max.   :108.127   Max.   : 1.771900   Max.   : 1.515600   Max.   : 5.04610  
       aT        
 Min.   :0.0360  
 1st Qu.:0.6270  
 Median :0.8670  
 Mean   :0.9261  
 3rd Qu.:1.1550  
 Max.   :5.2950  

### Data Aggregration and feature extraction
Here, the data is aggregated by 1 minute and features are extracted. Features are extracted to reduce the dimension of the data and only storing the representation of the data.

In [4]:
file1$group = as.factor(round(file1$time))
file2$group = as.factor(round(file2$time))
file3$group = as.factor(round(file3$time))
file4$group = as.factor(round(file4$time))
#(file1,20)

#list of all files
files = list(file1, file2, file3, file4)

#loop through all files and combine
features = NULL
for (i in 1:4){
res = files[[i]] %>%
    group_by(group) %>%
    summarize(ax_mean = mean(ax),
              ax_sd = sd(ax),
              ax_min = min(ax),
              ax_max = max(ax),
              ax_median = median(ax),
              ay_mean = mean(ay),
              ay_sd = sd(ay),
              ay_min = min(ay),
              ay_may = max(ay),
              ay_median = median(ay),
              az_mean = mean(az),
              az_sd = sd(az),
              az_min = min(az),
              az_maz = max(az),
              az_median = median(az),
              aT_mean = mean(aT),
              aT_sd = sd(aT),
              aT_min = min(aT),
              aT_maT = max(aT),
              aT_median = median(aT)
             )
    features = rbind(features, res)
}

#view all features
head(features)

group,ax_mean,ax_sd,ax_min,ax_max,ax_median,ay_mean,ay_sd,ay_min,ay_may,⋯,az_mean,az_sd,az_min,az_maz,az_median,aT_mean,aT_sd,aT_min,aT_maT,aT_median
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,-0.038164706,0.6594686,-1.2587,1.3821,-0.0955,-0.0682627451,0.7506785,-1.3892,1.6418,⋯,-0.13803333,0.9845115,-2.6753,2.7507,0.0254,1.273216,0.5830149,0.400,3.029,1.0770
1,-0.005806122,0.6325808,-1.6194,1.1943,-0.0015,0.0037908163,0.7819044,-1.5625,1.5428,⋯,-0.20496837,0.9252188,-3.0774,2.7158,-0.2121,1.263622,0.5448447,0.410,3.197,1.1375
2,0.069845455,0.6665500,-1.4554,1.4667,0.1070,0.0744333333,0.8022922,-1.4800,1.7951,⋯,-0.06405354,0.9293866,-1.8205,2.4862,-0.1512,1.298364,0.5131552,0.255,2.644,1.2830
3,0.011552525,0.5511310,-1.9254,1.2034,0.0675,0.0008262626,0.7894209,-2.0042,1.5577,⋯,-0.09287879,0.8893505,-2.1562,3.2355,-0.1672,1.203848,0.5125826,0.393,3.322,1.1180
4,0.046688119,0.6426574,-1.7805,1.4837,0.0836,-0.0177594059,0.7510811,-1.6629,1.4369,⋯,-0.13990000,0.9265720,-1.8515,3.5451,-0.1741,1.226267,0.5824608,0.313,3.597,1.1720
5,0.006678788,0.5780957,-1.4719,1.4355,0.0536,0.0013626263,0.7812245,-1.6293,1.6362,⋯,-0.16540000,0.9091516,-2.5561,2.9196,-0.2588,1.209515,0.5664847,0.336,3.035,1.1590


### Create Train and Test Set
To build an anomaly detection model, a train and test set is required. Here, the normal condition of the data is used for training and remaining is used for testing.

In [5]:
# create train and test set
train = features[1:67,2:ncol(features)]
test = features[68:nrow(features),2:ncol(features)]

trainLabels = rep(TRUE, 67)
testLabels = rep(FALSE, nrow(features)-67)

### One-Class SVM
Support vector machines (SVMs) are supervised learning models that analyze data and recognize patterns, and that can be used for both classification and regression tasks.

Typically, the SVM algorithm is given a set of training examples labeled as belonging to one of two classes. An SVM model is based on dividing the training sample points into separate categories by as wide a gap as possible, while penalizing training samples that fall on the wrong side of the gap. The SVM model then makes predictions by assigning points to one side of the gap or the other.

Sometimes oversampling is used to replicate the existing samples so that you can create a two-class model, but it is impossible to predict all the new patterns of fraud or system faults from limited examples. Moreover, collection of even limited examples can be expensive.

Therefore, in one-class SVM, the support vector model is trained on data that has only one class, which is the “normal” class. It infers the properties of normal cases and from these properties can predict which examples are unlike the normal examples. This is useful for anomaly detection because the scarcity of training examples is what defines anomalies: that is, typically there are very few examples of the network intrusion, fraud, or other anomalous behavior.

#### One-Class SVM using 'e1071'
Now we know what one-class svm is, we can build a model for anomaly detection. svm function from e1071 can be used to build the model. This model like anyother model does require model tuning. Here we are tuning the model at different parameters of gamma and nu values. Also, if you noticed we are scaling the data as well. To train the model, we will be using radial function. The summary of the trained model si as shown below. 

In [6]:
# train the model
model = svm(train, y = NULL, 
            scale = T, 
            type = 'one-classification', 
            na.action = na.omit, 
            kernel = 'radial',
            gamma = seq(0.01, 0.9, by = 0.01),
            nu = seq(0.01, 0.9, by=0.01)
           )

# view the trained model
model


Call:
svm.default(x = train, y = NULL, scale = T, type = "one-classification", 
    kernel = "radial", gamma = seq(0.01, 0.9, by = 0.01), nu = seq(0.01, 
        0.9, by = 0.01), na.action = na.omit)


Parameters:
   SVM-Type:  one-classification 
 SVM-Kernel:  radial 
      gamma:  0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1 0.11 0.12 0.13 0.14 0.15 0.16 0.17 0.18 0.19 0.2 0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3 0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4 0.41 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5 0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6 0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8 0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89 0.9 
         nu:  0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1 0.11 0.12 0.13 0.14 0.15 0.16 0.17 0.18 0.19 0.2 0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3 0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4 0.41 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5 0.51 0.52 0.53

Now, the model is trained, we can do the predictions with all the data we have to see how the model performed and then we can create a confusion matrix. The results are as shown below. 

In [7]:
# predict the data
predictions = predict(model, features[,2:ncol(features)])

# put data to a data frame
results = data.frame(prediction = as.factor(predictions),actual = as.factor(c(trainLabels,testLabels)))

# plot a confusion matrix
confusionMatrix(results$actual, results$prediction)

Confusion Matrix and Statistics

          Reference
Prediction FALSE TRUE
     FALSE   295    0
     TRUE      8   59
                                          
               Accuracy : 0.9779          
                 95% CI : (0.9569, 0.9904)
    No Information Rate : 0.837           
    P-Value [Acc > NIR] : < 2e-16         
                                          
                  Kappa : 0.9232          
                                          
 Mcnemar's Test P-Value : 0.01333         
                                          
            Sensitivity : 0.9736          
            Specificity : 1.0000          
         Pos Pred Value : 1.0000          
         Neg Pred Value : 0.8806          
             Prevalence : 0.8370          
         Detection Rate : 0.8149          
   Detection Prevalence : 0.8149          
      Balanced Accuracy : 0.9868          
                                          
       'Positive' Class : FALSE           
                     

### Session info
Below is the session info for the the packages and their versions used in this analysis. 

In [8]:
sessionInfo()

R version 3.3.3 (2017-03-06)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Debian GNU/Linux 9 (stretch)

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] caret_6.0-84    ggplot2_3.2.1   lattice_0.20-34 e1071_1.7-2    
[5] dplyr_0.8.3    

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-3        tidyselect_0.2.5    repr_1.0.1         
 [4] purrr_0.3.2         reshape2_1.4.3      splines_3.3.3      
 [7] vctrs_0.2.0         colorspace_1.4-1    generics_0.0.2     
[10] stats4_3.3.3        htmltools_0.3.6     base64enc_0.1-3    
[13] survival_2.40-1     prodlim_2018.04.18  rlang_0.4.0        
[